In [ ]:
import pandas as pd
import re
import nltk
from tqdm import tqdm
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import wordnet as wn
from functools import lru_cache
from sentence_transformers import SentenceTransformer

nltk.download('punkt')

loading + dropping irrelevant + merging the sentences back to their original

In [ ]:
trump = pd.read_excel("/Users/j/Documents/thesis/dyn.xlsx", sheet_name="trump")
clinton = pd.read_excel("/Users/j/Documents/thesis/dyn.xlsx", sheet_name="clinton")
df = pd.concat([trump, clinton], ignore_index=True)

df = df.loc[:, ~df.columns.str.startswith('SD_')]
df = df.iloc[:, 19:].reset_index(drop=True)
df['sentence'] = df['collocate left'].astype(str).fillna('') + ' ' + df['Kwic'].astype(str).fillna('') + ' ' + df['collocate right'].astype(str).fillna('')

df2 = df[['sentence', ' basic form  of  potential metaphorical word with word class', 'POS tag following CLAWS7 of COCA', 'metaphor type']]
df2['sentence'] = df2['sentence'].str.lower().fillna('')
df2

Umbrella terms (this code was not used in the final product, it needs refinement and improvement. It did reach somewhat of an estimation of the target domain)

In [ ]:
def get_umbrella_term(sentence, glove_embeddings, noun_boost=2.0):
    # Tokenize sentence
    words = word_tokenize(sentence.lower())
    
    # Remove stopwords and get POS tags
    stop_words = set(stopwords.words('english'))
    tagged_words = pos_tag([word for word in words if word.isalnum() and word not in stop_words])
    
    # Define relevant POS tags
    relevant_pos = {'NN', 'NNS', 'NNP', 'NNPS','VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ','JJ', 'JJR', 'JJS','RB', 'RBR', 'RBS'}
    relevant_words = [(word, pos) for word, pos in tagged_words if pos in relevant_pos]
    
    if not relevant_words:
        return None
    
    # Filter words that exist in GloVe embeddings
    valid_entries = [(word, pos) for word, pos in relevant_words if word in glove_embeddings]
    if not valid_entries:
        return None
    
    valid_words, valid_pos = zip(*valid_entries)
    # Get embeddings for valid words
    word_vectors = np.array([glove_embeddings[word] for word in valid_words])
    
    # Compute base attention weights based on vector norms
    norm_weights = np.linalg.norm(word_vectors, axis=1)
    
    # Apply POS-based weighting (boost nouns)
    pos_weights = np.array([noun_boost if pos in {'NN', 'NNS', 'NNP', 'NNPS'} else 1.0 
                           for pos in valid_pos])
    
    # Combine weights (norm * POS boost)
    weights = norm_weights * pos_weights
    weights = weights / weights.sum()  # Normalize to sum to 1
    
    # Calculate weighted mean vector
    weighted_mean = np.average(word_vectors, axis=0, weights=weights)
    
    # Calculate cosine similarity between weighted mean and each word vector
    similarities = {}
    for word, word_vector in zip(valid_words, word_vectors):
        similarity = np.dot(weighted_mean, word_vector) / (
            np.linalg.norm(weighted_mean) * np.linalg.norm(word_vector)
        )
        similarities[word] = similarity
    
    # Return word with highest similarity to the weighted mean
    if similarities:
        umbrella_term = max(similarities, key=similarities.get)
        return umbrella_term
    return None

# Apply the function with progress tracking
tqdm.pandas()  # Enable pandas integration with tqdm
df2['umbrella_term'] = df2['sentence'].progress_apply(
    lambda x: get_umbrella_term(x, glove_embeddings, noun_boost=2.0)
)

Preprocessing code; 1. fix for abreviations/contractions, 'doesnt' becomes 'does not' etc. 2. fix for 'Q' at start of sentence (for interview-like strings in the dataset). 3. Remove random numbers 4. remove other symbols

In [ ]:
# dictionary of common contractions
contractions_dict = {
    "doesn't": "does not",
    "isn't": "is not",
    "hasn't": "has not",
    "won't": "will not",
    "can't": "cannot",
    "shouldn't": "should not",
    "couldn't": "could not",
    "wouldn't": "would not",
    "don't": "do not",
    "didn't": "did not",
    "aren't": "are not",
    "weren't": "were not",
    "haven't": "have not",
    "hasn't": "has not",
    "isn't": "is not",
    "ain't": "is not",
}

def process_sentence(sentence):
    # remove Q symbol which appears at start of some sentences
    if sentence.startswith("q"):
        sentence = sentence[1:].strip()

    # Remove single number symbols
    sentence = re.sub(r'\b\d{1}\b', '', sentence)

    # Replace contractions with dict above. 
    words = sentence.split()
    sentence = ' '.join([contractions_dict.get(word, word) for word in words])

    # Remove other symbols (such as dot, question mark etc)
    sentence = re.sub(r'[^\w\s]', '', sentence)

    return sentence

tqdm.pandas()

# Apply the processing function to each sentence in the 'sentences' column
df2['processed_sentences'] = df2['sentence'].progress_apply(lambda x: process_sentence(str(x)) if x != 'nan' else x)

# Tokenize the sentences
df2['tokenized_sentences'] = df2['processed_sentences'].progress_apply(nltk.word_tokenize)


Loading glove embeddings

download the glove embeddings from: https://nlp.stanford.edu/projects/glove/

In [ ]:
# Load GloVe word vectors
def load_glove_embeddings(glove_file_path):
    embeddings_dict = {}
    with open(glove_file_path, 'r') as f:
        for line_num, line in enumerate(f, start=1):
            values = line.strip().split()
            if len(values) < 2:
                continue  
            try:
                word = values[0]
                vector = np.array(values[1:], dtype='float32')
                embeddings_dict[word] = vector
            except ValueError:
                print(f"Error: skipping line {line_num}.")
    print(f"succes (denk ik)")
    return embeddings_dict

glove_file = "/Users/j/Documents/thesis/glove.840B.300d.txt"
glove_embeddings = load_glove_embeddings(glove_file)

Compare GLOVE embeddings of words to embedding of  TOPIC word(s)


In [ ]:
## code will be added later ##

compare GLOVE - pairwise, all words vs all words. append mean similarity

In [ ]:
## code will be added later ##

compare WORDNET - TOPIC.

In [ ]:
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
from tqdm import tqdm
tqdm.pandas()

brown_ic = wordnet_ic.ic('ic-brown.dat')

# Similarity computation between 2 synsets with fallbacks
def compute_similarities(word_synset, topic_synset):
    similarities = {'WUP': None, 'PATH': None, 'LCH': None, 'JC': None, 'LIN': None}
    
    try:
        similarities['WUP'] = word_synset.wup_similarity(topic_synset)
    except:
        pass
    try:
        similarities['PATH'] = word_synset.path_similarity(topic_synset)
    except:
        pass
    try:
        similarities['LCH'] = word_synset.lch_similarity(topic_synset)
    except:
        pass
    try:
        similarities['JC'] = word_synset.jcn_similarity(topic_synset, brown_ic)
    except:
        pass
    try:
        similarities['LIN'] = word_synset.lin_similarity(topic_synset, brown_ic)
    except:
        pass
    
    return similarities

def compute_df_similarities(df):
    topic_word = df['umbrella_term'].iloc[0]
    topic_synset = get_first_noun_synset(topic_word)
    
    wup_list = []
    path_list = []
    lch_list = []
    jc_list = []
    lin_list = []
    
    for tokens in tqdm(df['tokenized_sentences'], desc="Computing WordNet Similarities"):
        wup_vals, path_vals, lch_vals, jc_vals, lin_vals = [], [], [], [], []

        for word in tokens:
            word_synset = get_first_noun_synset(word)
            if word_synset and topic_synset:
                sims = compute_similarities(word_synset, topic_synset)
                wup_vals.append(sims['WUP'] if sims['WUP'] is not None else 0)
                path_vals.append(sims['PATH'] if sims['PATH'] is not None else 0)
                lch_vals.append(sims['LCH'] if sims['LCH'] is not None else 0)
                jc_vals.append(sims['JC'] if sims['JC'] is not None else 0)
                lin_vals.append(sims['LIN'] if sims['LIN'] is not None else 0)
            else:
                wup_vals.append(0)
                path_vals.append(0)
                lch_vals.append(0)
                jc_vals.append(0)
                lin_vals.append(0)

        wup_list.append(wup_vals if any(wup_vals) else [sum(wup_vals)/len(wup_vals)])
        path_list.append(path_vals if any(path_vals) else [sum(path_vals)/len(path_vals)])
        lch_list.append(lch_vals if any(lch_vals) else [sum(lch_vals)/len(lch_vals)])
        jc_list.append(jc_vals if any(jc_vals) else [sum(jc_vals)/len(jc_vals)])
        lin_list.append(lin_vals if any(lin_vals) else [sum(lin_vals)/len(lin_vals)])

    df['wn_topic__WUP'] = wup_list
    df['wn_topic__PATH'] = path_list
    df['wn_topic__LCH'] = lch_list
    df['wn_topic__JC'] = jc_list
    df['wn_topic__LIN'] = lin_list
    return df


Compare WN pairwise

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
from tqdm import tqdm
from functools import lru_cache
import numpy as np

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('wordnet_ic')
brown_ic = wordnet_ic.ic('ic-brown.dat')

# Cache synset lookup
@lru_cache(maxsize=10000)
def get_first_noun_synset(word):
    synsets = wn.synsets(word, pos=wn.NOUN)
    return synsets[0] if synsets else None

# Cache similarity computation between synsets
@lru_cache(maxsize=100000)
def get_similarities_cached(word1, word2):
    s1 = get_first_noun_synset(word1)
    s2 = get_first_noun_synset(word2)
    if not s1 or not s2:
        return 0, 0, 0, 0, 0
    try:
        wp = s1.wup_similarity(s2) or 0
    except:
        wp = 0
    try:
        lch = s1.lch_similarity(s2) or 0
    except:
        lch = 0
    try:
        jc = s1.jcn_similarity(s2, brown_ic) or 0
    except:
        jc = 0
    try:
        lin = s1.lin_similarity(s2, brown_ic) or 0
    except:
        lin = 0
    try:
        path = s1.path_similarity(s2) or 0
    except:
        path = 0
    return wp, lch, jc, lin, path

# Main function
def compute_pairwise_similarities_fast(df):
    tqdm.pandas()

    def process(tokens):
        n = len(tokens)
        wp = np.zeros(n)
        lch = np.zeros(n)
        jc = np.zeros(n)
        lin = np.zeros(n)
        path = np.zeros(n)
        counts = np.zeros(n)

        for i in range(n):
            for j in range(i + 1, n):
                w1, w2 = tokens[i], tokens[j]
                sim_wp, sim_lch, sim_jc, sim_lin, sim_path = get_similarities_cached(w1, w2)

                wp[i] += sim_wp
                wp[j] += sim_wp
                lch[i] += sim_lch
                lch[j] += sim_lch
                jc[i] += sim_jc
                jc[j] += sim_jc
                lin[i] += sim_lin
                lin[j] += sim_lin
                path[i] += sim_path
                path[j] += sim_path
                counts[i] += 1
                counts[j] += 1

        # Avoiding error by /0 division
        wp = np.divide(wp, counts, out=np.zeros_like(wp), where=counts != 0)
        lch = np.divide(lch, counts, out=np.zeros_like(lch), where=counts != 0)
        jc = np.divide(jc, counts, out=np.zeros_like(jc), where=counts != 0)
        lin = np.divide(lin, counts, out=np.zeros_like(lin), where=counts != 0)
        path = np.divide(path, counts, out=np.zeros_like(path), where=counts != 0)

        return pd.Series([wp.tolist(), lch.tolist(), jc.tolist(), lin.tolist(), path.tolist()])

    df[['wn_pair__WP', 'wn_pair__LCH', 'wn_pair__JC', 'wn_pair__LIN', 'wn_pair__PATH']] = df['tokenized_sentences'].progress_apply(process)
    return df


Average similarity of glove (SEQUENTIALLY embedded)

In [ ]:
import ast

# Convert the stringified lists to actual lists
df['tokenized_sentences'] = df['tokenized_sentences'].apply(ast.literal_eval)

# Now apply the logic correctly
df['avg_sim_glove_repeated'] = df.apply(
    lambda row: [row['avg_sim_glove']] * len(row['tokenized_sentences']),
    axis=1
)


POS tagging sequentially (not used in paper)

In [ ]:
import pandas as pd
import spacy
from tqdm import tqdm

# Load spaCy model
nlp = spacy.load("en_core_web_sm")
pos_tags_list = []

for sentence in tqdm(df['sentence']):
    doc = nlp(sentence)
    pos_tags = [token.pos_ for token in doc]
    pos_tags_list.append(pos_tags)

df['pos_tags'] = pos_tags_list


Preprocessing target column (into 1 (metaphorical) and 0 (literal/non-metaphorical) words) The method of doing this spans multiple cells. 

In [ ]:
#conceptual metaphor column

In [ ]:
trump = pd.read_excel("/Users/j/Documents/thesis/dyn.xlsx", sheet_name="trump")
clinton = pd.read_excel("/Users/j/Documents/thesis/dyn.xlsx", sheet_name="clinton")
df = pd.concat([trump, clinton], ignore_index=True)

df = df.loc[:, ~df.columns.str.startswith('SD_')]
df = df.iloc[:, 19:].reset_index(drop=True)
df['sentence'] = df['collocate left'].astype(str).fillna('') + ' ' + df['Kwic'].astype(str).fillna('') + ' ' + df['collocate right'].astype(str).fillna('')

df2 = df[['sentence', 'conceptual metaphor']]
df2['sentence'] = df2['sentence'].str.lower().fillna('')
df2

In [ ]:
# save only between parenthesis (first word)

def extract_last_parentheses(text):
    if isinstance(text, str):
        matches = re.findall(r'\(([^()]*)\)', text)
        return matches[-1].strip().lower() if matches else None
    else:
        return text 

df['conceptual_metaphor'] = df['conceptual_metaphor'].apply(extract_last_parentheses)

df

In [ ]:
# if none, replace with word in basic_word_first .. . . 

In [ ]:
df['conceptual_metaphor'] = df['conceptual_metaphor'].fillna(df['basic_form_first_word'])

In [ ]:
# Get the index of the original column
col_index = df.columns.get_loc('conceptual_metaphor') + 1

# Insert the duplicate column
df.insert(col_index, 'conceptual_metaphor_copy', df['conceptual_metaphor'])

In [ ]:
#latere matching
import numpy as np

# mask: rows where conceptual_metaphor_copy is NaN & matched_token is not NaN
mask = df['conceptual_metaphor_copy'].isna() & df['matched_token'].notna()

# Fill conceptual_metaphor_copy with the first word of matched_token
df.loc[mask, 'conceptual_metaphor_copy'] = df.loc[mask, 'matched_token'].apply(
    lambda x: str(x).strip().split()[0] if str(x).strip() else np.nan
)


In [ ]:
#some abstract code to extract metaphorical word from col.

col_name = ' basic form  of  potential metaphorical word with word class'

def clean_and_split(text):
    if pd.isna(text) or text.strip() == "":
        return None
    no_parentheses = re.sub(r'\([^)]*\)', '', text)
    word_list = no_parentheses.strip().split()
    return word_list

# Apply the function
df['word_list'] = df[col_name].apply(clean_and_split)


In [ ]:
df['conceptual_metaphor_copy'] = df['conceptual_metaphor_copy'].apply(
    lambda x: x.split() if isinstance(x, str) else x
)

make sure the word form is the same as in the sentence. So if the stem is noted, find the right form in the actual sentence and adjust accordingly.

In [ ]:
import pandas as pd
import re
from rapidfuzz import fuzz
from tqdm import tqdm

# Enable tqdm for pandas
tqdm.pandas()

#get word(s) from the 'metaphor' col
def parse_metaphor(metaphor_str):
    metaphor_str = str(metaphor_str)
    match = re.search(r'\[(.*?)\]', metaphor_str)
    if match:
        content = match.group(1)
        words = [re.sub(r'[^a-zA-Z]', '', word.strip()) for word in content.split(',')]
        return [w for w in words if w]
    return []

#find most similar words in the tokenized sentence.
def find_sequential_match(sentence_words, target_words):
    if not target_words or not sentence_words:
        return []
    
    matched_words = []
    search_start = 0
    
    for target in target_words:
        best_score = -1
        best_word = None
        best_index = -1
        
        for i in range(search_start, len(sentence_words)):
            score = fuzz.ratio(target, sentence_words[i])
            if score > best_score:
                best_score = score
                best_word = sentence_words[i]
                best_index = i
        
        if best_word is not None:
            matched_words.append(best_word)
            search_start = best_index + 1
        else:
            matched_words.append(None)
    
    return matched_words

df['matched_words'] = df.progress_apply(
    lambda row: find_sequential_match(row['tokenized_sentences'], parse_metaphor(row['conceptual_metaphor_copy'])),
    axis=1
)

print(df)

The final code to compare matched_words with the words in tokenized_sentence; it makes a new sequential column, where metaphorical words are annotated with a 1 and nonmetaphorical words are annotated with a 0.   

In [ ]:
def process_row_for_pattern(row):
    sentence_raw = row['tokenized_sentences']
    pattern_raw = row['matched_words']

    #As between sessions i loaded from a saved excell sheet sometimes the types were ruined dwhich therefore had to be fixed
    try:
        sentence = ast.literal_eval(sentence_raw) if isinstance(sentence_raw, str) else sentence_raw
        pattern = ast.literal_eval(pattern_raw) if isinstance(pattern_raw, str) else pattern_raw

    if not isinstance(sentence, list) or not isinstance(pattern, list):
        print(f"Invalid input types in row {row.name}: sentence={type(sentence)}, pattern={type(pattern)}")
        return [0] * len(sentence) if isinstance(sentence, list) else []

    #actual code
    if not sentence or not pattern:
        return [0] * len(sentence) if sentence else []

    result = [0] * len(sentence)
    pattern_idx = 0  # Tracks the next word to match in pattern

    for i, token in enumerate(sentence):
        if pattern_idx < len(pattern) and token == pattern[pattern_idx]:
            result[i] = 1
            pattern_idx += 1

    return result

df['match_result'] = df.apply(process_row_for_pattern, axis=1)

Below the code for the training of the models

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Configuration and determinism 
MODEL_NAME = 'bert-large-uncased'
BATCH_SIZE = 8
MAX_LEN = 128
NUM_EPOCHS = 20
PATIENCE = 3
LEARNING_RATE = 2e-5

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class MetaphorDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=MAX_LEN):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        tokens = row['tokenized_sentences']
        labels = row['match_result']

        encoding = self.tokenizer(
            tokens,
            is_split_into_words=True,
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=self.max_len
        )

        word_ids = encoding.word_ids()
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        aligned_labels = []
        for i, word_id in enumerate(word_ids):
            if word_id is None:
                aligned_labels.append(-100)
            else:
                try:
                    aligned_labels.append(labels[word_id])
                except IndexError:
                    aligned_labels.append(-100)

        labels_tensor = torch.tensor(aligned_labels, dtype=torch.long)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels_tensor
        }

# Model
class MetaphorClassifier(nn.Module):
    def __init__(self, model_name=MODEL_NAME):
        super(MetaphorClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        hidden_dim = self.bert.config.hidden_size
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(hidden_dim, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        seq_output = self.dropout(outputs.last_hidden_state)
        logits = self.classifier(seq_output)
        return logits

# splitting
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = MetaphorDataset(train_df, tokenizer)
val_dataset = MetaphorDataset(val_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# Initialize model on GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MetaphorClassifier().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

# Early stopping
best_val_loss = float('inf')
counter = 0

# Training loop
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = loss_fn(logits.view(-1, 2), labels.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            logits = model(input_ids, attention_mask)
            loss = loss_fn(logits.view(-1, 2), labels.view(-1))
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), 'best_model.pt')
        counter = 0
    else:
        counter += 1
        if counter >= PATIENCE:
            print("Early stopping triggered.")
            break

# Evaluation
model.load_state_dict(torch.load('best_model.pt'))
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        logits = model(input_ids, attention_mask)
        preds = torch.argmax(logits, dim=-1).view(-1).cpu().numpy()
        labels = labels.view(-1).cpu().numpy()

        mask = labels != -100
        all_preds.extend(preds[mask])
        all_labels.extend(labels[mask])

print(classification_report(all_labels, all_preds, digits=4))


the same exact code was ran with the other models, replace MODEL_NAME with the following.

In [ ]:
#from the transformer library.
MODEL_NAME = 'bert-large-uncased'
MODEL_NAME = 'bert-base-uncased'
MODEL_NAME = 'microsoft/deberta-v3-base'
MODEL_NAME = 'microsoft/deberta-v3-large'
MODEL_NAME = 'roberta-large'
MODEL_NAME = 'roberta-base'


MODEL_NAME = 'roberta-base' #used in combination with the roberta tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', add_prefix_space=True)

Code for ablation

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm
import numpy as np
import random

# Feature columns
FEATURE_COLUMNS = [
    'wn_pair__WP', 'wn_pair__LCH', 'wn_pair__JC', 'wn_pair__LIN', 'wn_pair__PATH',
    'wn_topic__WUP', 'wn_topic__LCH', 'wn_topic__JC', 'wn_topic__LIN', 'wn_topic__PATH', 'glove'
]

# determinism
MODEL_NAME = 'microsoft/deberta-v3-base'
MAX_LEN = 128
BATCH_SIZE = 8
NUM_EPOCHS = 20
PATIENCE = 3
LEARNING_RATE = 2e-5

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class MetaphorDataset(Dataset):
    def __init__(self, df, tokenizer, feature_name, max_len=MAX_LEN):
        self.feature_name = feature_name
        self.labels = df['match_result'].tolist()
        self.features = df[feature_name].tolist()

        self.inputs = tokenizer(
            df['tokenized_sentences'].tolist(),
            is_split_into_words=True,
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=max_len
        )

        # Save word_ids separately
        self.word_ids = [
            self.inputs.word_ids(batch_index=i)
            for i in range(len(self.labels))
        ]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        input_ids = self.inputs['input_ids'][idx]
        attention_mask = self.inputs['attention_mask'][idx]
        word_ids = self.word_ids[idx]

        feature_vecs = []
        aligned_labels = []

        for i, word_id in enumerate(word_ids):
            if word_id is None:
                feature_vecs.append([0.0])
                aligned_labels.append(-100)
            else:
                try:
                    val = self.features[idx][word_id]
                    feature_vecs.append([float(val)])
                    aligned_labels.append(self.labels[idx][word_id])
                except Exception:
                    feature_vecs.append([0.0])
                    aligned_labels.append(-100)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'features': torch.tensor(feature_vecs, dtype=torch.float32),
            'labels': torch.tensor(aligned_labels, dtype=torch.long)
        }

# Model
class MetaphorClassifier(nn.Module):
    def __init__(self, feature_dim=1, model_name=MODEL_NAME):
        super(MetaphorClassifier, self).__init__()
        self.deberta = AutoModel.from_pretrained(model_name)
        hidden_dim = self.deberta.config.hidden_size

        self.feature_proj = nn.Linear(feature_dim, 64)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(hidden_dim + 64, 2)

    def forward(self, input_ids, attention_mask, features):
        outputs = self.deberta(input_ids=input_ids, attention_mask=attention_mask)
        seq_output = outputs.last_hidden_state

        projected_feats = self.feature_proj(features)
        combined = torch.cat([seq_output, projected_feats], dim=-1)
        combined = self.dropout(combined)
        logits = self.classifier(combined)
        return logits

# Run ablation study
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scaler = torch.cuda.amp.GradScaler()

for feature in FEATURE_COLUMNS:
    print(f"\{feature}")
    set_seed(42)

    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
    train_dataset = MetaphorDataset(train_df, tokenizer, feature)
    val_dataset = MetaphorDataset(val_df, tokenizer, feature)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, pin_memory=True)

    model = MetaphorClassifier().to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

    best_val_loss = float('inf')
    counter = 0

    for epoch in range(NUM_EPOCHS):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Training [{feature}] Epoch {epoch+1}"):
            input_ids = batch['input_ids'].to(device, non_blocking=True)
            attention_mask = batch['attention_mask'].to(device, non_blocking=True)
            features = batch['features'].to(device, non_blocking=True)
            labels = batch['labels'].to(device, non_blocking=True)

            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                logits = model(input_ids, attention_mask, features)
                logits = logits.view(-1, 2)
                labels = labels.view(-1)
                loss = loss_fn(logits, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Validation [{feature}]"):
                input_ids = batch['input_ids'].to(device, non_blocking=True)
                attention_mask = batch['attention_mask'].to(device, non_blocking=True)
                features = batch['features'].to(device, non_blocking=True)
                labels = batch['labels'].to(device, non_blocking=True)

                with torch.cuda.amp.autocast():
                    logits = model(input_ids, attention_mask, features)
                    logits = logits.view(-1, 2)
                    labels = labels.view(-1)
                    loss = loss_fn(logits, labels)

                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f} | Val Loss = {avg_val_loss:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), f'best_model_{feature}.pt')
            counter = 0
        else:
            counter += 1
            if counter >= PATIENCE:
                print("Early stopping.")
                break

    # Evaluation
    model.load_state_dict(torch.load(f'best_model_{feature}.pt'))
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Evaluating [{feature}]"):
            input_ids = batch['input_ids'].to(device, non_blocking=True)
            attention_mask = batch['attention_mask'].to(device, non_blocking=True)
            features = batch['features'].to(device, non_blocking=True)
            labels = batch['labels'].to(device, non_blocking=True)

            with torch.cuda.amp.autocast():
                logits = model(input_ids, attention_mask, features)

            preds = torch.argmax(logits, dim=-1).view(-1).cpu().numpy()
            labels = labels.view(-1).cpu().numpy()
            mask = labels != -100
            all_preds.extend(preds[mask])
            all_labels.extend(labels[mask])

    print(f"\nClassification Report for feature: {feature}")
    print(classification_report(all_labels, all_preds, digits=4))
